## Data Clean-up and Pre-Processing 

- Brian K
- Edited: 10/25/2019

In [1]:
import pandas as pd
import numpy as np
from collections import *
import string

# Custom Code imports
import sys
sys.path.insert(0, '/Users/briankalinowski/PycharmProjects/FakeNewsChallenge/Code')
import DataProcessing as pro

# All news


In [2]:
news_data = pd.read_csv('/Users/briankalinowski/Desktop/Data/General_News/all_the_news.csv')
news_data.drop(columns=['year', 'month'], inplace=True)
news_data.date = pd.to_datetime(news_data.date)
news_data.head()

,id,title,publication,author,date,url,content
0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,NaN,WASHINGTON — Congressional Republicans have...
1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,NaN,"After the bullet shells get counted, the blood..."
2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,NaN,"Death may be the great equalizer, but it isn’t..."
4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [3]:
fill_vals = {'title': 'NO_TITLE', 
             'author': 'NO_AUTHOR', 
             'url': 'NO_URL'
            }

news_data = pro.fill_null_string_columns(news_data, fill_vals)

In [4]:
news_content = news_data[['title', 'content']]
news_content.head()

,title,content
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."


In [5]:
news_content['content_length'] = news_content.content.str.len()

/anaconda/envs/FakeNewsChallenge/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
news_content = news_content.sort_values(by=['content_length'], ascending=False)
news_content= news_content.reset_index(drop=True)
news_content.head()

,title,content,content_length
0,Full transcript: FBI Director James Comey test...,Below is the full transcript from the...,292586
1,\nWill the Liberal Bubble Burst? Your Thoughts\n,Atlantic readers from across the political spe...,164658
2,Full transcript: Sally Yates and James Clapper...,Former acting attorney general Sally...,162120
3,\nTake It From a ‘Liberal Redneck’\n,Atlantic readers from across the political ...,162104
4,\nTrying to Avoid Politics Over Thanksgiving\n,Atlantic readers from across the political spe...,152708


In [7]:
news_content = pro.clean_text_column(news_content, 'title')

In [8]:
news_content = pro.clean_text_column(news_content, 'content')

In [10]:
news_content.head()

,title,content,content_length
0,Full transcript FBI Director James Comey testi...,Below is the full transcript from the...,292586
1,Will the Liberal Bubble Burst? Your Thoughts,Atlantic readers from across the political spe...,164658
2,Full transcript Sally Yates and James Clapper ...,Former acting attorney general Sally...,162120
3,Take It From a Liberal Redneck’,Atlantic readers from across the political ...,162104
4,Trying to Avoid Politics Over Thanksgiving,Atlantic readers from across the political spe...,152708


## Fake news 

In [2]:
fake_news = pd.read_csv('/Users/briankalinowski/Desktop/Data/Kaggle/fake_news.csv')
fake_news.drop(columns=['uuid', 'ord_in_thread', 'published'], inplace=True)
fake_news = fake_news.dropna(subset=['text'])

In [3]:
fake_news.head()

,author,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,Barracuda Brigade,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,reasoning with facts,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,Barracuda Brigade,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,Fed Up,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,Fed Up,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [8]:
fake_news_content = fake_news[['title', 'text', 'type']]
fake_news_content = fake_news_content.reset_index(drop=True)
fake_news_content.head()

,title,text,type
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,bias
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,bias
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,bias
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,bias
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,bias


In [12]:
fake_news_content.title = fake_news_content.title.apply(lambda row: str(row).lower())
fake_news_content.text = fake_news_content.text.apply(lambda row: str(row).lower())

In [13]:
fake_news_content = pro.clean_text_column(fake_news_content, 'title')
fake_news_content = pro.clean_text_column(fake_news_content, 'text')

In [14]:
fake_news_content.head()

,title,text,type
0,muslims busted they stole millions in gov’t be...,print they should pay all the back all the mon...,bias
1,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,bias
2,breaking weiner cooperating with fbi on hillar...,red state fox news sunday reported this morni...,bias
3,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,bias
4,fantastic! trumps 7 point plan to reform healt...,email healthcare reform to make america great ...,bias


In [19]:
# list(fake_news_content[fake_news_content.type == 'fake']['text'])